In [7]:
import pandas as pd
import numpy as np
import datetime as dt

In [8]:
extracted = pd.read_excel(r'data/extracted_data.xlsx')
correction = pd.read_excel(r'data/agent_correction_requests.xlsx')

data = extracted.copy()
corrected = correction.copy()

In [9]:
class Data:
    def __init__(self, df1, df2):
        self.df1 = df1
        self.df2 = df2
    def clean_reference(self) -> pd.DataFrame:
        self.df1['Account Reference'] = self.df1['Account Reference'].str.replace('\'','')
        return self.df1

    def split_text(self) -> pd.DataFrame:
        notes = self.df1[self.df1['Notes'].str.len() > 100]
        notes.loc[:, 'Notes'] = notes.Notes.str[100:]
        self.df1 = pd.concat([self.df1, notes])
        self.df1.Notes = self.df1.Notes.str[0:101]
        self.df1 = self.df1.sort_index().reset_index(drop=True)
        return self.df1

    def clean_data(self) -> pd.DataFrame:
        cleaned = self.df1.copy()
        corr = self.df2.copy().astype(str)
        corr['Date'] = corr['Date'] + " " + corr['Time']
        cleaned = cleaned[(cleaned['Date Gathered'].isin(corr['Date'])) & (cleaned['Account Reference'].isin(corr['Account number']))]

        try:
            for i in range(len(cleaned)):
                cleaned[corr.iloc[i,4]].iloc[i] = corr.iloc[i,5]
            self.df1.update(cleaned)
            self.df1.drop_duplicates(inplace=True, ignore_index=True)
        except FutureWarning:
            pass
        return self.df1

    def ib_arrangement(self, cleaned: pd.DataFrame):
        cleaned: pd.DataFrame = cleaned[(cleaned['PTP Date'] != "0000-00-00 00:00:00") & (cleaned['PTP Amount'] > 0) & (cleaned['Disposition Desc'] == 'PROMISE TO PAY')]

        cleaned: pd.DataFrame = pd.DataFrame({
            'Account Reference': cleaned['Account Reference'],
            'Instalment Date': cleaned['PTP Date'],
            'Arrangement Amount': cleaned['PTP Amount'],
            'Instalment Paid Value': "0",
            'Instalment Paid Date': "",
            'Settlement Program': "PTP",
            'With Post Dated Check': "N"
        })
        cleaned['Arrangement Amount'] = cleaned['Arrangement Amount'].apply(lambda x: "{:.2f}".format(x))
        cleaned['Instalment Date'] = pd.to_datetime(cleaned['Instalment Date']).dt.strftime('%m/%d/%Y')
        cleaned.reset_index(drop=True, inplace=True)
        return cleaned


    def correction_status(self) -> pd.DataFrame:
        cleaned = self.df1.copy()
        self.df2 = self.df2.astype(str)
        self.df2['Date'] = self.df2['Date'] + " " + self.df2['Time']
        cleaned = cleaned[(cleaned['Date Gathered'].isin(self.df2['Date'])) & (cleaned['Account Reference'].isin(self.df2['Account number']))]
        self.df2['Status'] = np.where(self.df2['Account number'].isin(cleaned['Account Reference']), "Corrected", "Not Found")
        return self.df2

    def save_data(self, main_1: pd.DataFrame, main_2: pd.DataFrame, file_2: pd.DataFrame):
        main_file: str = input('Main File Name: ')
        second_file: str = input('Second File Name: ')
        yesterday = (dt.date.today() - dt.timedelta(1)).strftime('%m%d%Y')

        with pd.ExcelWriter(f'{main_file}_{yesterday}.xlsx') as writer:
            main_1.to_excel(writer, sheet_name='All Data' ,index=False)
            main_2.to_excel(writer, sheet_name='IB Arrangement' ,index=False)

        with pd.ExcelWriter(f'{second_file}_{yesterday}.xlsx') as writer:
            file_2.to_excel(writer, sheet_name='IB Arrangement' ,index=False)


In [ ]:
if __name__ == '__main__':
    output = Data(data, corrected)
    output.clean_reference()
    output.split_text()
    output.save_data(output.clean_data(), output.ib_arrangement(output.clean_data()), output.correction_status())

In [11]:
sample = Data(data,corrected)
sample.ib_arrangement(sample.clean_data())

,Account Reference,Instalment Date,Arrangement Amount,Instalment Paid Value,Instalment Paid Date,Settlement Program,With Post Dated Check
0,'OXXXXXXXO8PXXQO9',09/12/2024,13600.00,0,,PTP,N
1,'JYYYYYYYKJT6DKJ6',09/16/2024,10000.00,0,,PTP,N
2,'JYYYYYYYKJJ6DFLL',09/12/2024,870.00,0,,PTP,N
3,'OXXXXXXXO8QUTOSX',09/12/2024,4500.00,0,,PTP,N
4,'JYYYYYYYKFDY6DFD',09/16/2024,6000.00,0,,PTP,N
...,...,...,...,...,...,...,...
57,'10000000BEDD0E10',09/12/2024,4034.44,0,,PTP,N
58,'100000000FDG1HIH',09/16/2024,31500.00,0,,PTP,N
59,'100000001DBIEB1D',09/16/2024,10600.00,0,,PTP,N
60,'100000001DBF01FH',09/12/2024,7900.00,0,,PTP,N
